In [2]:
#My default packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from dataclasses import dataclass
import itertools
import heapq


# Day 7

In [24]:
class File:
    def __init__(self,name :str, parent =None,size :int =0) -> None:
        self.name =name
        self.parent = parent
        self._size = size

    def __repr__(self) -> str:
        return f"File({self.name} in {self.parent.name if self.parent else None} (size={self._size}))"

    def size(self):
        return self._size


class Dir(File):
    def __init__(self,name :str, parent=None) -> None:
        File.__init__(self,name,parent)

        self.contains = {}

    def __repr__(self) -> str:
        return f"Dir({self.name} in {self.parent.name if self.parent else None} (contains {len(self.contains)} elements)"

    def size(self):
        return sum(e.size() for e in self.contains.values())
    def add(self,elem):
        self.contains[elem.name] = elem


In [25]:
root = Dir("/")
with open("input7.txt") as f:
    while line:= f.readline().rstrip():
        if line[:4]=="$ cd":
            destination = line.split(" ")[-1]
            if destination=="..":
                curdir=curdir.parent
            elif destination=="/":
                curdir = root
            else:
                curdir=curdir.contains[destination]
        elif line[:4]=="$ ls":
            continue
        else:
            sd,name = line.split(" ")
            if sd=="dir":
                curdir.add(Dir(name,curdir))
            else:
                sd= int(sd)
                curdir.add(File(name,curdir,sd))




In [26]:
def looksmalldirs(dir,maxsize=100000,smaldir=None):
    if smaldir is None:
        smaldir = set()
    if dir.size()<=maxsize:
        smaldir.add(dir)
    for subdir in dir.contains.values():
        if type(subdir) is Dir:
            smaldir = looksmalldirs(subdir,maxsize,smaldir)
    return smaldir
def lookbigdirs(dir,minsize=100000,smaldir=None):
    if smaldir is None:
        smaldir = set()
    if dir.size()>=minsize:
        smaldir.add(dir)
    for subdir in dir.contains.values():
        if type(subdir) is Dir:
            smaldir = lookbigdirs(subdir,minsize,smaldir)
    return smaldir

In [27]:
smoldir = looksmalldirs(root,maxsize=100000)
sum(map(Dir.size,smoldir))

1783610

In [28]:
totaldiskspace = 70_000_000
neededunusedspace = 30000000

todelete = neededunusedspace-(totaldiskspace-root.size())

bigdir = lookbigdirs(root,minsize=todelete)
min(map(Dir.size,bigdir))

4370655